# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 1.0119e+00   valLoss:9.6275e-01  time: 1.34e+00
epoch: 1   trainLoss: 9.6522e-01   valLoss:9.5938e-01  time: 1.03e+00
epoch: 2   trainLoss: 8.1773e-01   valLoss:9.6351e-01  time: 1.01e+00
epoch: 3   trainLoss: 8.3811e-01   valLoss:9.8133e-01  time: 9.97e-01
epoch: 4   trainLoss: 7.6172e-01   valLoss:9.5356e-01  time: 9.99e-01
epoch: 5   trainLoss: 7.3495e-01   valLoss:9.3944e-01  time: 9.99e-01
epoch: 6   trainLoss: 7.5432e-01   valLoss:9.1543e-01  time: 1.02e+00
epoch: 7   trainLoss: 7.0806e-01   valLoss:8.7357e-01  time: 1.02e+00
epoch: 8   trainLoss: 6.5537e-01   valLoss:7.8877e-01  time: 1.00e+00
epoch: 9   trainLoss: 6.8540e-01   valLoss:7.1905e-01  time: 1.01e+00
epoch: 10   trainLoss: 6.5779e-01   valLoss:6.7026e-01  time: 1.02e+00
epoch: 11   trainLoss: 6.1681e-01   valLoss:6.4414e-01  time: 1.01e+00
epoch: 12   trainLoss: 5.9811e-01   valLoss:6.2438e-01  time: 9.99e-01
epoch: 13   trainLoss: 5.5766e-01   valLoss:6.0480e-01  time: 1.02e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000020,0.003031,0.134033,0.856558,0.945003,0.109992,-12.437497
test,0.000033,0.003552,0.151230,0.652423,0.885501,0.024699,-11.094243


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

alt.HConcatChart(...)